---

(🎧) 표시가 되어 있는 부분은 Wire 페이지에서 오디오 코드 해설을 제공하고 있습니다.

---

# 실습 2. CBOW 학습하기

지난 실습에서는 위키백과 문단 코퍼스를 사용해 단어사전을 만들었습니다.   
이번 모듈에서는 CBOW 알고리즘을 사용해, 우리 단어사전에 있는 각각의 토큰 임베딩 벡터를 학습시켜보겠습니다.   


<b>학습 목표:    
- CBOW 알고리즘 구조를 이해한다.
- TensorFlow 2.0을 이용해 CBOW를 학습하고, 학습이 완료된 워드 벡터를 추출한다.</b>







## #0. 실습 준비하기
지난 모듈에서 저장한 단어사전과 데이터를 로드하고,   
이번 시간에 학습한 워드 벡터를 저장하기 위해 구글 드라이브를 마운트하겠습니다.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#### > 라이브러리 설치

Colab은 런타임에 연결될 때마다 로컬 환경이 초기화됩니다.   
따라서 한국어 형태소 분석 패키지 konlpy를 설치하고, tokenize 함수를 정의하겠습니다.

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 3.8MB 51.6MB/s 
     |████████████████████████████████| 92kB 8.6MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


<font color = "red">[MISSION]    
적절한 형태소 분석기를 이용해    
문장을 인풋으로 받아 형태소 분석된 리스트를 반환하는 tokenize 함수를 구현해보세요</font>  

Hint: 지난 모듈에서 Komoran, Hannanum, Kkma, Okt 중 어떤 형태소 분석기를 사용했나요?

In [ ]:
from konlpy.tag import Komoran, Hannanum, Kkma, Okt
komoran = Komoran()
 
def tokenize(sentence):
  """ Your Code Here """
 
  return komoran.morphs(sentence)
 
tokenize("미션 완료!!")

['미션', '완료', '!!']

## #1. 학습 데이터 준비하기
이제 CBOW 학습을 위한 데이터를 준비하겠습니다.   
NLU 모델링을 위해서는 태스크에 적합한 인풋 데이터와 라벨을 준비해야 합니다.   
이 과정에서 토크나이징과 인코딩이 필요합니다. 

#### Step 1. 토크나이징



<img src = "https://github.com/seungyounglim/temporary/blob/master/fig_step1.PNG?raw=true">

먼저 학습 데이터와 검증 데이터에 있는 문단을 tokenize 함수를 이용해 토크나이징해두겠습니다.

In [ ]:
## 구글 드라이브에 있는 데이터셋 로딩
import json
with open("/content/gdrive/My Drive/NLP/CBOW_train_paras.json" , 'r') as f:
  PARAS_tr = json.loads(f.read())
  
with open("/content/gdrive/My Drive/NLP/CBOW_dev_paras.json" , 'r') as f:
  PARAS_dev = json.loads(f.read())

print("Train: {} | Val: {}".format(len(PARAS_tr), len(PARAS_dev)))

Train: 8000 | Val: 1681


<font color = "red">[MISSION]    
PARAS_tr과 PARAS_dev에 있는 각각의 문장을 tokenize한 후,    
각각 TOKEN_PARAS_tr와 TOKEN_PARAS_dev리스트에 append하세요</font>

Hint: 실행 완료된 후 TOKEN_PARAS_tr ->
- [ [토큰1, 토큰2, ...] , [토큰1, 토큰2, ...] ] 형태
- 리스트의 아이템 수 len(TOKEN_PARAS_tr) = 8000
- 각 아이템은 문장에 있는 토큰 수 만큼의 길이를 가짐

In [ ]:
# 토크나이징된 문장을 각각 TOKEN_PARAS_tr, TOKEN_PARAS_dev에 저장
from tqdm import tqdm
 
TOKEN_PARAS_tr = []
TOKEN_PARAS_dev = []
 
""" Your Code Here """
for para in tqdm(PARAS_tr):
  tokenized_para = tokenize(para)
  TOKEN_PARAS_tr.append(tokenized_para)
 
TOKEN_PARAS_dev = []
for para in tqdm(PARAS_dev):
  tokenized_para = tokenize(para)
  TOKEN_PARAS_dev.append(tokenized_para)

100%|██████████| 1681/1681 [00:16<00:00, 103.90it/s]


#### Step 2. 인코딩   
토크나이징이 완료된 문단은 모델이 처리할 수 있는 의미단위인 토큰으로 나뉘었습니다.   
그런데 아직 모델이 인식할 수 있는 정수의 형태가 아닙니다.   
따라서 지난 시간에 만들어놓은 단어사전을 이용해 자연어 문장을 정수 인덱스로 바꾸는 기능을 코딩하겠습니다.   


In [ ]:
## 구글 드라이브에 있는 단어 리스트 로딩
with open("/content/gdrive/My Drive/NLP/meta.tsv") as f:
  vocabulary_list = [v.strip() for v in f.readlines()]
print("토큰 개수: {}개".format(vocabulary_list))

토큰 개수: ['[PAD]', '[UNK]', '하', '이', '.', '의', '는', '을', '다', 'ㄴ', '에', ',', '았', '은', '를', '었', '고', '되', '아', '에서', '가', '으로', '있', '들', '로', ')', '어', '(', '년', '과', '적', '도', 'ㄹ', '"', '지', '것', '와', 'ㄴ다', '기', '그', '게', '수', "'", '등', '던', '며', '으며', '않', '대하', '에게', '만', '1', '받', '》', '《', '2', '는데', '라고', '없', '지만', '위하', '까지', '주', '한', '면서', '다고', '말', '중', '·', '때', '라는', '명', '3', '이후', '면', '나', '아서', '다는', '일', '자', '전', '번', '같', '으나', '부터', '보', '때문', '또한', '두', '크', '라', '위', '자신', '시키', '내', '미국', '후', '개', '4', '따르', '성', '오', '많', 'ㅁ', '당시', '시작', '제', '5', '아니', '기록', '가지', '사람', '그러나', '-', '시', '다른', '통하', '더', '보이', '대', '지역', '의하', '함께', '세', '못하', '“', '일본', '”', '만들', '경기', '‘', '’', '및', '그리고', '가장', '사용', '〈', '〉', '보다', '주장', '정부', '경우', '째', '상', '활동', '어서', '뒤', '화', '밝히', '다시', '이나', '팀', '6', '세계', '시즌', '대통령', '문제', ':', '모두', '점', '첫', '여러', '영화', '안', '이러', '한국', '간', '처음', '모든', '국가', '쓰', '아야', '곡', '사실', '%', '10', '동안', '알', '나오', '가능', '생각', '정치

In [ ]:
vocabulary_list[:10]

['[PAD]', '[UNK]', '하', '이', '.', '의', '는', '을', '다', 'ㄴ']

<font color = "red">[MISSION] 단어사전을 이용해 토큰을 정수로 바꾸는 Python Class 만들기 </font>

TextEncoder   
  - @convert_tokens_to_ids: 토큰 리스트를 인풋으로 받아 정수 인덱스의 리스트로 리턴
  - @convert_ids_to_tokens: 인덱스 리스트를 인풋으로 받아 원본 토큰의 리스트로 리턴
  


우리는 TextEncoder라는 클래스를 정의하여 단어 리스트를 인풋으로 받아    
토큰-> 인덱스 변환과 인덱스 -> 토큰 변환을 수행하는 기능을 수행할 수 있도록 하겠습니다. 


아래 코드를 보고, 빈 칸을 채워넣어 이 두 가지 기능을 할 수 있는 TextEncoder 클래스를 완성하세요.

In [ ]:
class TextEncoder(object):
    def __init__(self, vocab_list): # vocab_list를 받아서 기능을 수행함
 
      self.pad_token = "[PAD]" # 딥러닝 패딩 처리를 위한 토큰 명시
      self.oov_token = "[UNK]" # Unknown 토큰 처리하는 토큰 명시
 
      ## 토큰을 인덱스로 바꾸는 dictionary 만들기
      token_to_id = {}
 
      """
      [★CODE 1★]
        - vocab_list에 있는 토큰을 순서대로 token_to_id에 추가하세요.
          (예) "[PAD]"  : 0
               "[UNK]"  : 1
               "하"  : 2
               "이" : 3
               "." : 4
      """
 
      """ ------- 여기에 코드를 작성하세요 -------"""
 
      for i, token in enumerate(vocab_list):
        token_to_id[token] = i
 
      """ ----------------------------------------"""
 
      ## 인덱스를 토큰으로 바꾸는 dictionary 만들기
      ids_to_tokens = {v:k for k,v in token_to_id.items()} 
 
      self.token_to_id = token_to_id
      self.ids_to_tokens = ids_to_tokens
      self.vocab_size = len(token_to_id)  
      self.vocab_list = vocab_list
     
    def convert_tokens_to_ids(self, tokens):
      ids = []
 
      for token in tokens:
        if token in self.token_to_id:
          """mission: 토큰을 token_to_id에서 찾을 수 있으면 해당 인덱스를 ids에 append"""
          ids.append(self.token_to_id[token]) ## [★CODE 2★]
        else:
          """mission: 토큰을 token_to_id에서 찾을 수 없으면 [UNK]에 해당하는 인덱스를 append"""
          ids.append(self.token_to_id[self.oov_token]) ## [★CODE 3★]
      return ids
 
    def convert_ids_to_tokens(self, ids):
      return [self.ids_to_tokens[i] for i in ids]

우리 단어사전 리스트인 vocabulary_list를 인풋으로 넣어 text_encoder 클래스를 만들겠습니다.  
이렇게 한 번 코딩을 해 놓으면, 단어 사전이 바뀌어도 같은 코드를 복사해 사용할 수 있어 편리합니다 :)

In [ ]:
text_encoder = TextEncoder(vocabulary_list)

In [ ]:
print("PAD 토큰 확인:", text_encoder.pad_token)
print("OOV 토큰 확인:", text_encoder.oov_token)
print("단어사전 크기:", text_encoder.vocab_size)

PAD 토큰 확인: [PAD]
OOV 토큰 확인: [UNK]
단어사전 크기: 70002


이제 만든 함수가 정상적으로 작동하는지, 기능을 테스트해보겠습니다.

In [ ]:
## 단어사전을 이용해 정수 인덱스로 바꾸기
sent = "이 문장을 파싱해서 정수로 바꿔보겠음."
tokenized_sent = tokenize(sent)
print(tokenized_sent)

text_encoder.convert_tokens_to_ids(tokenized_sent)

['이', '문장', '을', '파', '싱', '해서', '정수', '로', '바꾸', '어', '보', '겠', '음', '.']


[3, 2796, 7, 563, 5419, 5921, 13018, 24, 785, 26, 85, 192, 214, 4]

In [ ]:
## 정수 인덱스를 토큰으로 바꾸기
sent = "거꾸로 돌리기"
tokenized_sent = tokenize(sent)
print(tokenized_sent)

ids = text_encoder.convert_tokens_to_ids(tokenized_sent)
text_encoder.convert_ids_to_tokens(ids)

['거꾸로', '돌리', '기']


['거꾸로', '돌리', '기']

그럼 우리가 가진 코퍼스를 방금 정의한 text_encoder를 사용해 모델이 인식할 수 있는 인덱스로 바꾸어 저장하겠습니다.   

<font color = "red">[MISSION]    
text_encoder를 사용해 TOKEN_PARAS_tr과 TOKEN_PARAS_dev에 있는 토크나이징된 문단을 정수 인덱스로 바꾸고   
TOKEN_IDS_tr과 TOKEN_IDS_dev로 저장하세요 </font>

In [ ]:
## Train Data
TOKEN_IDS_tr = []
TOKEN_IDS_dev = []
 
""" Your Code Here """
for sent in TOKEN_PARAS_tr:
  TOKEN_IDS_tr.append(text_encoder.convert_tokens_to_ids(sent))
 
for sent in TOKEN_PARAS_dev:
  TOKEN_IDS_dev.append(text_encoder.convert_tokens_to_ids(sent))

In [ ]:
## 코드가 정상적으로 작동했는지 확인하기
print("Index Example:", TOKEN_IDS_tr[0])
print(text_encoder.convert_ids_to_tokens(TOKEN_IDS_tr[0]))

Index Example: [1042, 2501, 9759, 22, 6, 3518, 10, 50, 22, 8, 4, 3770, 24, 1157, 17, 6, 2454, 2024, 7904, 3577, 20, 51, 71, 22, 16, 2103, 7, 19526, 2, 56, 11, 1268, 10, 4432, 449, 2103, 7, 62, 74, 1746, 10, 2749, 26, 34, 9, 3029, 12867, 5, 243, 14, 209, 26, 62, 16, 2064, 20, 1468, 37, 4, 7904, 20, 209, 26, 62, 9, 3029, 12867, 19, 7564, 29, 11940, 7, 513, 51, 97, 917, 233, 7, 41, 22, 8, 4, 1061, 34, 47, 569, 511, 724, 554, 146, 257, 7904, 6, 11941, 26, 330, 26, 39, 3632, 14, 1825, 32, 41, 22, 8, 4, 7904, 20, 338, 9, 146, 1670, 3519, 14, 515, 18, 15410, 6, 111, 3, 101, 6, 2981, 20, 2982, 30, 21, 268, 2, 37, 4, 7904, 6, 743, 15, 7, 69, 233, 7, 41, 22, 6, 5478, 3, 1208, 20, 213, 34, 47, 38, 86, 10, 17, 212, 1061, 6, 35, 3, 5479, 8, 4, 11942, 7904, 49, 2103, 7, 62, 15, 7, 69, 11, 7904, 20, 1881, 669, 36, 122, 10394, 57, 2, 56, 11, 3, 69, 4330, 74, 1918, 10, 2699, 16, 3194, 709, 6, 141, 20, 22, 56, 11, 1377, 39, 24, 1157, 17, 37, 4, 3708, 29, 11943, 13, 58, 59, 1158, 3268, 36, 3072, 3, 324, 

#### Step 3. CBOW 학습 가능한 형태로 변환하기
우리가 가진 데이터는 위키피디아 문단을 토크나이징한 데이터입니다.   

하지만 CBOW 학습을 위해서는 예측하고자 하는 토큰 앞뒤로 w개의 토큰을 가지고 와야 하지요.   
예를 들어 w = 3으로 설정했다면, 각각의 train example은   
- X : [(t-3째 토큰), (t-2째 토큰), (t-1째 토큰), (t+1째 토큰), (t+2째 토큰), (t+3째 토큰)]    
- Y : [(t번째 토큰)]   
의 형태여야 합니다.    

아래의 generate_context_word_pairs 함수는 토크나이징된 코퍼스와 윈도우 크기를 인풋으로 받아 이러한 형태로 데이터를 만드는 역할을 합니다.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
def generate_context_word_pairs(corpus, window_size = 3):
  """
  index로 바뀐 코퍼스 리스트를 인풋으로 받아
  CBOW 학습용 input과 label을 리턴하는 함수
  """
  inputs = []
  labels = []

  context_length = window_size*2
  for words in corpus:
    sentence_length = len(words)
    for index, word in enumerate(words):
      if index < window_size or index >= len(words)- window_size:
        # window size 안에 만들 수 없는 것들은 만들지 않음.
        continue

      context_words = []            
      start = index - window_size
      end = index + window_size + 1
      
      context_words= [words[i] 
                            for i in range(start, end) 
                            if 0 <= i < sentence_length 
                            and i != index]                      

      assert(len(context_words) == context_length)
      inputs.append(context_words)
      labels.append(word)

  return inputs, labels

윈도우 크기를 3으로 설정하고, 데이터를 만들겠습니다.   
- inputs_tr의 각 example은 길이가 6 (앞뒤로 3개, 총 6개 토큰),   
- labels_tr의 각 example은 길이가 1 (맞춰야 하는 가운데 토큰)   
의 형태가 됩니다.   

validation 데이터도 마찬가지로 만들어줍니다.

In [ ]:
import numpy as np

WINDOW_SIZE = 3
inputs_tr, labels_tr = generate_context_word_pairs(TOKEN_IDS_tr, window_size=WINDOW_SIZE)
inputs_dev, labels_dev = generate_context_word_pairs(TOKEN_IDS_dev, window_size=WINDOW_SIZE)


텐서플로우에서는 numpy array 형태의 인풋을 받기 때문에 np.array 형태로 데이터를 바꾸겠습니다.

In [ ]:
inputs_tr = np.array(inputs_tr)
labels_tr = np.array(labels_tr)
inputs_dev = np.array(inputs_dev)
labels_dev = np.array(labels_dev)

In [ ]:
print("TRAIN:", inputs_tr.shape, labels_tr.shape)
print("VAL  :", inputs_dev.shape, labels_dev.shape)

TRAIN: (2013892, 6) (2013892,)
VAL  : (420084, 6) (420084,)


👉 우리는 학습 코퍼스 8000문단을 가지고 있었는데요,    
    각 문단을 윈도우를 주며 데이터를 만들었더니 총 2,013,892건의 많은 데이터셋이 만들어졌습니다.   
    
👉 이렇게 Unsupervised Learning 방식으로 쉽게 모을 수 있는 raw 코퍼스를 이용해 워드 벡터 학습에 사용할 수 있습니다.

In [ ]:
### CBOW 예시
print("** Example**")
for i in range(10):
  input_tokens = text_encoder.convert_ids_to_tokens(inputs_tr[i])
  gt_token = text_encoder.convert_ids_to_tokens([labels_tr[i]])[0]
  print("{} ___ {} -> {}".format(" ".join(input_tokens[:WINDOW_SIZE])," ".join(input_tokens[WINDOW_SIZE:]), gt_token))

** Example**
차량 정비 소가 ___ 는 패턴 에 -> 있
정비 소가 있 ___ 패턴 에 만 -> 는
소가 있 는 ___ 에 만 있 -> 패턴
있 는 패턴 ___ 만 있 다 -> 에
는 패턴 에 ___ 있 다 . -> 만
패턴 에 만 ___ 다 . 화가 -> 있
에 만 있 ___ . 화가 로 -> 다
만 있 다 ___ 화가 로 추정 -> .
있 다 . ___ 로 추정 되 -> 화가
다 . 화가 ___ 추정 되 는 -> 로


## #2. CBOW 학습하기

<img src = "https://github.com/seungyounglim/temporary/blob/master/fig_step3+4.PNG?raw=true">


#### Step 1. 모델 구조 만들기
이제 데이터(인풋, 라벨)가 준비되었으니 본격적으로 모델링을 할 차례입니다.   
keras의 Sequential 함수를 사용해 CBOW 모델 구조를 만들겠습니다.


<font color = "red">[MISSION] Sequential을 이용해 CBOW 모델을 만들어보세요! </font>

👉 Summary가 아래와 같아야 함

```
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
embedding (Embedding)        (None, 6, 128)            8960256   
_________________________________________________________________
lambda (Lambda)              (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 70002)             9030258   
=================================================================
Total params: 17,990,514
Trainable params: 17,990,514
Non-trainable params: 0
_________________________________________________________________
None
```






In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda
 
VOCAB_SIZE = text_encoder.vocab_size ## 단어사전 개수
EMBED_SIZE = 128 ## 임베딩 차원 개수
INPUT_LENGTH = WINDOW_SIZE * 2 ## 인풋 길이
 
 
cbow_model = Sequential()
# 1. 임베딩 레이어 추가
cbow_model.add( Embedding(VOCAB_SIZE, EMBED_SIZE, input_length = INPUT_LENGTH) ) ## [★CODE 1★]
 
# 2. 임베딩된 벡터들의 평균 구하기 (HINT의 Lambda 레이어 함수 사용)
cbow_model.add(Lambda(lambda x: tf.keras.backend.mean(x, axis=1), output_shape=(EMBED_SIZE,)))
 
# 3. 가운데 들어갈 단어를 예측하는 Fully Connected Layer 연결
cbow_model.add(Dense(VOCAB_SIZE, activation = "softmax")) ## [★CODE 2★]
 
print(cbow_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 6, 128)            8960256   
_________________________________________________________________
lambda (Lambda)              (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 70002)             9030258   
Total params: 17,990,514
Trainable params: 17,990,514
Non-trainable params: 0
_________________________________________________________________
None


#### Step 2. 모델 컴파일하기

<font color = "red">[MISSION] 적당한 Loss 함수와 optimizer을 지정해 모델을 컴파일하세요!</font>

In [ ]:
cbow_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

#### Step 3. 모델 학습하기

<b>Callback 정의하기</b>    

TensorFlow에서는 Callback을 정의해서 학습하는동안 모델을 체크할 수 있습니다.   

대표적인 예시로 EarlyStoppingCallback을 사용하면 validation loss를 트래킹하다가   
 validation loss가 증가하면 학습을 알아서 멈춥니다.   

여기서는 CustomCallback을 정의해서 CBOW 모델을 확인하도록 하겠습니다.   

우리가 이 모델을 학습하는    
이유는 <u>자연어 의미상 비슷한 단어를 비슷한 의미공간상에 매핑하기 위함</u>이었습니다.   

학습이 진행되는동안, valid_dataset에 있는 각각의 명사 토큰에 대해 가장 가깝게 임베딩된 토큰이 어떤 것들이 있는지 확인할 것입니다.    
이를 통해 정말 관련 있는 토큰들이 비슷한 공간에 매핑되는지 확인하겠습니다. 

In [ ]:
class MyCustomCallback(tf.keras.callbacks.Callback):

  """샘플 단어와 가장 유사한 N개 단어를 보여줌"""
  reverse_dictionary = text_encoder.ids_to_tokens

  def on_epoch_end(self, batch, logs=None):
    valid_dataset = ["사과", "취업", "밥", "사랑", "수학", "운동", "가수", "회사", "자신감"]
    valid_dataset = text_encoder.convert_tokens_to_ids(valid_dataset)

    embedding = cbow_model.get_weights()[0]

    reverse_dictionary = text_encoder.ids_to_tokens
    norm = tf.keras.backend.sqrt(tf.reduce_sum(tf.keras.backend.square(embedding), 1, keepdims=True))
    normalized_embeddings = embedding / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    print("")
    for val_i in range(len(valid_dataset)):
      valid_word = reverse_dictionary[valid_dataset[val_i]]
      top_k = 8 # number of nearest neighbors
      nearest = np.array(-similarity[val_i, :]).argsort()[1:top_k+1] 
      print("{} -> {}".format(valid_word, " , ".join(text_encoder.convert_ids_to_tokens(nearest))))

model.fit() 매서드를 통해 학습을 진행하겠습니다. (🎧)     

위키 백과의 일부 문단만 사용했는데도 학습이 완료되기까지 약 25분이 소요됩니다.   
일은 GPU에게 시키고, 학습이 진행되는동안 좀 쉬다 올까요😋

In [ ]:
history = cbow_model.fit(inputs_tr, labels_tr, epochs = 3, batch_size=256,
               validation_data = (inputs_dev, labels_dev),
               callbacks = [MyCustomCallback()])

# 학습 완료된 임베딩 저장하기
final_embeddings = cbow_model.get_weights()[0]
final_embeddings = np.array(final_embeddings)

import io
out_v = io.open("/content/gdrive/My Drive/NLP/vecs.tsv" , 'w', encoding='utf-8')

for num, word in enumerate( text_encoder.vocab_list):
  vec = final_embeddings[num]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()

Epoch 1/3
7867/7867 [==============================] - ETA: 0s - loss: 6.3229
사과 -> 환영 , 조달 , 압박 , 대우 , 항의 , 규탄 , 변호 , 비난
취업 -> 농담 , 불특정 , 환상 , 황종희 , 감성 , 동정 , 자유의 여신상 , 스폰서
밥 -> 밧줄 , 딛 , 휠체어 , 머리카락 , 사도왕후 , 잡아먹 , 불어넣 , 루피
사랑 -> 동작 , 형벌 , 견제 , 상호작용 , 구원 , 상속 , 존중 , 수련
수학 -> 본질 , 주기 , 해부학 , 계획 경제 , 근원 , 의사소통 , 야학 , 삼투
운동 -> 정책 , 세력 , 개혁 , 권력 , 시스템 , 시설 , 기관 , 교육
가수 -> 전문가 , 수감자 , 아티스트 , 음악가 , 남자 , 장교 , 중국인 , 대표자
회사 -> 청소년 , 이슬람 , 음악가 , 처 , 작곡가 , 도시 , 원로 , 상어
자신감 -> 굴욕 , 인지도 , 충성심 , 두려움 , 제물 , 섞 , 묘안 , 물건
7867/7867 [==============================] - 488s 62ms/step - loss: 6.3229 - val_loss: 5.5683
Epoch 2/3
7867/7867 [==============================] - ETA: 0s - loss: 5.4215
사과 -> 규탄 , 변호 , 의논 , 환영 , 발송 , 이입 , 풍문 , 애도
취업 -> 반항 , 중재자 , 외지 , 타이밍 , 주의력 , 오토 폰 비스마르크 , 광학 망원경 , 찜질방
밥 -> 뒷짐 , 휠체어 , 포위망 , 함석 , 왕관 , 수정란 , 불쾌감 , 시공사
사랑 -> 존경 , 형벌 , 존중 , 자랑 , 충성 , 살육 , 배달 , 상속
수학 -> 문학 , 철학 , 해부학 , 신학 , 도덕 , 화학 , 예술 , 문법
운동 -> 정책 , 개혁 , 기관 , 세력 , 사업 , 교육 , 시스템 , 대책
가수 -> 아티스트 , 선수 , 여자 , BABY VOX I

## #3. 결과 확인하기
이제 학습 완료된 결과물을 확인하며, 비슷하게 매핑된 단어를 눈으로 확인해보겠습니다. 

In [ ]:
## 런타임 다시 연결했을 경우 구글 드라이브 마운팅
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
## 저장한 단어사전과 워드벡터 로딩
import numpy as np
with open("/content/gdrive/My Drive/NLP/vecs.tsv") as f:
  vecs = [v.strip() for v in f.readlines()]
final_embeddings = [v.split("\t") for v in vecs]
final_embeddings = np.array(final_embeddings, dtype="float32")
with open("/content/gdrive/My Drive/NLP/meta.tsv") as f:
  meta = [v.strip() for v in f.readlines()]
meta[0]

'[PAD]'

CustomCallback에서 벡터상에 가장 가까이 위치한 토큰을 찾아왔건 것처럼   
토큰을 입력하면 가장 비슷하게 매핑된 단어를 찾아오는 search_nearest 함수를 정의했습니다.   
원하는 토큰을 넣어서 비슷한 토큰이 어떤 것이 매핑되었는지 테스트해보아요. 

In [ ]:
import tensorflow as tf

reverse_dictionary = {}
token_to_id_dictionary = {}
for i, token in enumerate(meta):
  reverse_dictionary[i] = token
  token_to_id_dictionary[token] = i

def search_nearest(search_token, top_k = 5):
  if search_token not in token_to_id_dictionary:
    print("해당 토큰은 단어사전에서 찾을 수 없음")
  search_id = token_to_id_dictionary[search_token]
  print("{} -> {}".format(search_token, search_id))

  norm = tf.keras.backend.sqrt(tf.reduce_sum(tf.keras.backend.square(final_embeddings), 1, keepdims=True))
  normalized_embeddings = final_embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, [search_id])
  similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
  print("")

  nearest = np.array(-similarity[0, :]).argsort()[1:top_k+1] 
  print("Nearest Tokens: {}".format(" , ".join([reverse_dictionary[s] for s in nearest])))

In [ ]:
search_nearest("학교")

학교 -> 368

Nearest Tokens: 부산대학교 , 대학교 , 약대 , 군포 , 대구대학교


In [ ]:
search_nearest("어머니")

어머니 -> 557

Nearest Tokens: 아버지 , 아내 , 할아버지 , 딸 , 오빠


In [ ]:
search_nearest("이명박")

이명박 -> 2370

Nearest Tokens: 노무현 , 이승만 , 문재인 , 한나라당 , 김대중


In [ ]:
search_nearest("철학")

철학 -> 1241

Nearest Tokens: 문학 , 신학 , 과학 , 예술 , 음악
